In [1]:
#Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
#SPDX-License-Identifier: MIT-0

In [2]:
#install additional libraries
!pip install nltk
!pip install jsonlines
!pip install pandarallel
!pip install tensorflow==2.1
!pip install --upgrade grpcio 
!pip install --upgrade s3fs

  Using cached jsonlines-2.0.0-py3-none-any.whl (6.3 kB)
  Using cached pandarallel-1.5.2-py3-none-any.whl
     |██████████████████████▎         | 293.0 MB 113.3 MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |███████████████████████████████▎| 412.1 MB 109.6 MB/s eta 0:00:01

     |███████████████████████████████▊| 417.9 MB 109.6 MB/s eta 0:00:01

     |████████████████████████████████| 421.8 MB 109.6 MB/s eta 0:00:01

     |████████████████████████████████| 421.8 MB 20 kB/s 
     |████████████████████████████████| 3.8 MB 116.8 MB/s eta 0:00:01
     |████████████████████████████████| 448 kB 114.2 MB/s eta 0:00:01
     |████████████████████████████████| 26.1 MB 53.4 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 120.9 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 127.7 MB/s eta 0:00:01
     |████████████████████████████████| 146 kB 128.6 MB/s eta 0:00:01
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.3
    Uninstalling scipy-1.5.3:
      Successfully uninstalled 

     |████████████████████████████████| 4.2 MB 19.9 MB/s 


  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.31.0
    Uninstalling grpcio-1.31.0:
      Successfully uninstalled grpcio-1.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-serving-api 1.15.0 requires tensorflow~=1.15.0, but you have tensorflow 2.1.0 which is incompatible.
tensorflow-cpu 1.15.5 requires tensorboard<1.16.0,>=1.15.0, but you have tensorboard 2.1.1 which is incompatible.
tensorflow-cpu 1.15.5 requires tensorflow-estimator==1.15.1, but you have tensorflow-estimator 2.1.0 which is incompatible.
     |████████████████████████████████| 115 kB 67.9 MB/s eta 0:00:01
     |████████████████████████████████| 7.4 MB 62.9 MB/s eta 0:00:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.20.99
    Uninstalling botocore-1.20.99:
      Successfully uninstalled botocore-1.20.99
  Attempting uni

In [4]:
#import libraries
import os 
import json
import shutil
import tensorflow as tf
import pandas as pd
import numpy as np

import boto3
import sagemaker
import nltk

from search_utils import helpers, search_preprocessing

# 1. Deploy a SageMaker Endpoint

In [21]:
#Creating a sagemaker session
sagemaker_session = sagemaker.Session()

#We'll be using the sagemaker default bucket
#Feel free to change this to another bucket name and make sure it's the same across all four notebooks
bucket_name = "sagemaker-knn-benfelip"

#Copy the glove_job_name, this was generated automatically in step 3 of the training notebook
glove_job_name = "search-glove-2106302153111456"

#Copy the training_job_name, this was generated automatically in step 4 of the training notebook
training_job_name = "search-training-2106302201087225"

In [23]:
sagemaker_session.create_model_from_job(training_job_name=training_job_name,
                          env={'INFERENCE_PREFERRED_MODE': 'embedding'})

endpoint_config_name = sagemaker_session.create_endpoint_config(name=training_job_name,
                                                      model_name=training_job_name,
                                                      initial_instance_count=1,
                                                      instance_type='ml.m4.xlarge')

#Specify the name of the endpoint
endpoint_name = "object2vec-embeddings2"

sagemaker_session.create_endpoint(endpoint_name=endpoint_name, config_name=training_job_name, tags=None, wait=True)

INFO:sagemaker:Creating model with name: search-training-2106302201087225
INFO:sagemaker:Creating endpoint-config with name search-training-2106302201087225


ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: Cannot create already existing endpoint configuration "arn:aws:sagemaker:us-east-1:062241367565:endpoint-config/search-training-2106302201087225".

# 2. Generate predictions using the SageMaker Endpoint

In [24]:
#Reading the data from S3 and loading the vocabulary 
bucket_name="sagemaker-knn-benfelip"
data = pd.read_csv(f"s3://{bucket_name}/search_knn_blog/data/processed_data/data.csv")

word_to_id = helpers.read_json_from_s3(bucket_name,\
                                       f'search_knn_blog/sagemaker-runs/{glove_job_name}/vocab.json')

In [25]:
#Sample a few products from the overall catalog
sub_set = data.sample(10000)
descriptions = sub_set["processed_title"]

In [26]:
from sagemaker.predictor import json_serializer, json_deserializer

# define encode-decode format for inference data
predictor = sagemaker.predictor.RealTimePredictor(endpoint_name)
#predictor.content_type = 'application/json'
predictor.serializer = json_serializer
predictor.deserializer = json_deserializer
tokenizer = nltk.tokenize.TreebankWordTokenizer()

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [27]:
def l2_normalize(v):
    """
    This functions normalise the embeddings based on the L2 norm.
    """
    
    norm = np.sqrt(np.sum(np.square(v)))

    return v / norm

In [ ]:
#This is the "enc_dim" parameter you have set in the training job hyperparameters of object2vec
#By default this value is set to 512 in the training notebook
embedding_size=512

In [28]:
all_embeddings, labels = [], []
for i, description in enumerate(descriptions):
    if i%1000==0:
        print(f"Processing product {i}/{len(descriptions)}")

    enc_description = search_preprocessing.sentence_to_integers(description, tokenizer, word_to_id)
    if len(enc_description) != 0:
        payload = {"instances" : [{"in0": enc_description}]}
        result = predictor.predict(payload)
        embeddings = result["predictions"][0]["embeddings"]
        embeddings = l2_normalize(embeddings)
        labels.append(sub_set.iloc[i]["product_category"])
        all_embeddings.append(embeddings)
    else:
        all_embeddings.append([0]*embedding_size)
        labels.append(sub_set.iloc[i]["product_category"])


See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Processing product 0/10000


See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs

...

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs

In [30]:
#Transforming predictions to "float64" numpy array
labels = np.array(labels, dtype="str")

X = []
for em_value in all_embeddings:
    X.append(em_value)
    
embeddings = np.array(X)
embeddings.dtype = "float64"
print(embeddings.shape)

(10000, 512)


# 3. Visualise the embeddings using Tensorboard projector

In [31]:
#Create a directory for storing tensorboard logs
#If a directory exists (previous runs) make sure to clean it up
if os.path.isdir("../tensorboard_logs"):
    shutil.rmtree("../tensorboard_logs")
    os.mkdir("../tensorboard_logs")
else:
    os.mkdir("../tensorboard_logs")

In [32]:
from tensorboard.plugins import projector

def register_embedding(embedding_tensor_name, metadata_path, logs_dir):
    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    embedding.tensor_name = embedding_tensor_name
    embedding.metadata_path = metadata_path
    projector.visualize_embeddings(logs_dir, config)
    
#Setting the tensorboard logs directory and additional variables
logs_dir = '../tensorboard_logs' 
metadata_path = 'metadata.tsv' 
embedding_tensor_name = 'embeddings'
EMBEDDINGS_FPATH = os.path.join(logs_dir, f'{embedding_tensor_name}.ckpt')

#Registering and saving the embeddings in the logs directory
tf.compat.v1.reset_default_graph()
register_embedding(embedding_tensor_name, metadata_path, logs_dir)
tf.compat.v1.disable_eager_execution()
tensor_embeddings = tf.Variable(embeddings, name=embedding_tensor_name)
tf_session = tf.compat.v1.InteractiveSession()
tf_session.run(tf.compat.v1.global_variables_initializer())
saver = tf.compat.v1.train.Saver()
saver.save(tf_session,  os.path.join(logs_dir, f'{embedding_tensor_name}.ckpt'), 0)
tf_session.close()

#Saving the labels
with open(os.path.join(logs_dir, metadata_path), 'w') as f:
    for label in labels:
        f.write('{}\n'.format(label))
            


# 4. Open Tensorboard

# 5. Saving enriched data for Elasticsearch 

In [33]:
str_all_embeddings = [str(list(e)) for e in all_embeddings]
sub_set["embeddings"]= str_all_embeddings

In [34]:
def row2dict(x):
    x = x.dropna().to_dict()

    for key in x:
        x[key] = str(x[key])


    return x

records_to_save = sub_set.apply(lambda x: row2dict(x), axis=1)
records_to_save = list(records_to_save.values)


with open("./data.json", "w") as write_file:
    json.dump(records_to_save, write_file)



boto3.client("s3").upload_file("./data.json",
                           bucket_name, "search_knn_blog/data/enriched_data/data.json")

# 5. Delete the endpoint (Unless you plan to continue to notebook 4)

In [35]:
#Make sure you delete your endpoint when you're done making predictions
#sagemaker_session.delete_endpoint(endpoint_name="object2vec-embeddings")